## Dependencies

In [1]:
import re
import warnings
from joblib import dump
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape, GlobalAveragePooling1D, LSTM, Conv1D, SpatialDropout1D
from tensorflow.keras.callbacks import ReduceLROnPlateau
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
model_path = '/kaggle/input/59-googleq-a-train-use-2xlstm/use.h5'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

## Test set

In [5]:
# Test features
X_test_title = test['question_title']
X_test_body = test['question_body']
X_test_answer = test['answer']

X_test = [X_test_title, X_test_body, X_test_answer]

# Model parameters

In [6]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)

# Model

In [7]:
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

def encoder_block(input_layer):
    encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
    return encoder

In [8]:
# Title
input_title = Input(shape=(1,), dtype=tf.string, name='input_title')
encoded_title = encoder_block(input_title)

title = Reshape((EMBEDDDING_SIZE, 1))(encoded_title)
title = LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(title)
title_out = LSTM(64, kernel_initializer='lecun_normal', recurrent_dropout=0.5)(title)

# Body
input_body = Input(shape=(1,), dtype=tf.string, name='input_body')
encoded_body = encoder_block(input_body)

body = Reshape((EMBEDDDING_SIZE, 1))(encoded_body)
body = LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(body)
body_out = LSTM(64, kernel_initializer='lecun_normal', recurrent_dropout=0.5)(body)

# Answer
input_answer = Input(shape=(1,), dtype=tf.string, name='input_answer')
encoded_answer = encoder_block(input_answer)

answer = Reshape((EMBEDDDING_SIZE, 1))(encoded_answer)
answer = LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(answer)
answer_out = LSTM(64, kernel_initializer='lecun_normal', recurrent_dropout=0.5)(answer)


# Output
x = Concatenate()([encoded_title, encoded_body, encoded_answer, title_out, encoded_body, answer_out])
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_initializer='lecun_normal')(x)
x = Dropout(0.5)(x)
output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='lecun_normal', name='output')(x)
model = Model(inputs=[input_title, input_body, input_answer], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_answer (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512)          0           input_title[0][0]                
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 512)          0           input_answer[0][0]               
______________________________________________________________________________________________

# Make predictions

In [9]:
Y_test = model.predict(X_test)

In [10]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.909788,0.600967,0.266588,0.627674,0.566983,0.622617,0.638268,0.594743,0.343964,...,0.883547,0.915158,0.630926,0.957980,0.961317,0.835596,0.151377,0.095229,0.768196,0.915499
1,46,0.879718,0.535760,0.003842,0.741396,0.856178,0.908522,0.572360,0.465017,0.183318,...,0.721505,0.922534,0.616633,0.959849,0.962708,0.832157,0.876457,0.098108,0.130111,0.858511
2,70,0.880528,0.632227,0.048916,0.741212,0.867094,0.884987,0.590313,0.488607,0.355828,...,0.839758,0.913662,0.600179,0.951059,0.948649,0.817315,0.104208,0.123328,0.760485,0.874537
3,132,0.843053,0.440803,0.010596,0.662721,0.773605,0.884813,0.526456,0.431494,0.295992,...,0.699461,0.917153,0.656067,0.955934,0.968297,0.847871,0.759519,0.142942,0.718646,0.886752
4,200,0.960541,0.582894,0.032646,0.886144,0.753066,0.893218,0.685604,0.670557,0.327886,...,0.814923,0.966962,0.696079,0.985350,0.987278,0.906051,0.131293,0.106814,0.875379,0.944502
